In [13]:
import pandas as pd
import random 
import itertools
import json
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_rows', 50)


from functions import sharpe_ratio_calculation, generate_rand_portfolios, select_top_five, join_stocks_crypto, distance_matrix_calc, run_min_variance, run_clustering_model, test_for_silhouette_score

In [14]:
df_all_stocks = pd.read_csv('stocks_data_filled.csv',index_col='Date')
#df_all_stocks = df_all_stocks.bfill()

#df_all_stocks.to_csv('stocks_data_filled.csv')

cryptos_df = pd.read_csv('cryptos_data.csv', index_col='Date')


joined_df = join_stocks_crypto(cryptos_df, df_all_stocks, mode = 'stocks_left') #mode - either do left with crypto and fill NA for stocks or do left on stocks and leave out some dates for cryptos
joined_df.index = pd.to_datetime(joined_df.index)
joined_df = joined_df.bfill()


joined_df_weekly = joined_df.resample('W').last() #try aggregating on a weekly level
joined_df_3days = joined_df.resample('3D').last()# aggregating on a twice per week basis to arrive at the sweet spot of that 250 (1 year) timeseries length

In [15]:
#Random Portfolios generation
tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)

In [16]:
#Select top five sharpe ratio portfolios from a portfolio
sharpe_ratio = sharpe_ratio_calculation(df_all_stocks, rf_rate_annual = 0.02)
top_five_dict = select_top_five(random_portfolios, metric=sharpe_ratio)

In [17]:
#Optimize Traditional Portfolios
# for i in range(999, 1001):

#     print('Doing', i)

#     top_five_sets = dict(itertools.islice(top_five_dict.items(), i, i+1))
#     results = run_min_variance(df_all_stocks, top_five_sets, min_weight_for_top_five=0.05)  #TRY DIFFERENT WEIGHTS FOR top_five
#     with open(f"min_variance_portfolio_jsons/my_dict{i}.json", "w") as f:
#         json.dump(results, f, indent=4)

In [18]:
#Reassemble the results of the optimization - jsons
# min_var_portfolios = dict()
# for i in range(1,1000):
#     with open(f'min_variance_portfolio_jsons/my_dict{i}.json') as f:
#         port = json.load(f)
#         min_var_portfolios.update(port)

# with open(f"full_optimized_min_variance.json", "w") as f:
#     json.dump(min_var_portfolios, f, indent=4)

In [19]:
n_clusters_list = [4,5,6,7]
linkage_list=['single', 'average', 'complete']
window_sizes = [3,7,10,14,21,30,60]


def run_clustering_evaluation(df, window_sizes, method, moving_average=True, return_mode='arithmetic', df_input_name='DFWASNOTSPECIFIED'):

    # df_input_name = input('Put in the name of the df mode you are running for: ')

    for w_size in window_sizes:

        #return_mode = 'arithmetic'
        #n_init = 3
        #center = True
        if moving_average:
            df = df.rolling(window=w_size, center=True).mean()

            smoothing = 'moving_average'
        else:
            smoothing = 'no_smoothing'

        if len(df) < 150 and w_size > 30:
            continue
        silhouette_df = test_for_silhouette_score(df, n_clusters_list, method=method, return_mode=return_mode, n_init=3, linkage_list=linkage_list)

        silhouette_df['return_mode'] = 'arithmetic'
        silhouette_df['n_init'] = 3
        silhouette_df['smoothing'] = smoothing
        silhouette_df['window_size/span'] = w_size

        silhouette_df.to_csv(f'silhouette_dfs/{method}_{smoothing}_{return_mode}_windowsize-{w_size}_{df_input_name}.csv')




df_input_name = input('Put in the name of the df mode you are running for: ')
for return_mode in ['arithmetic', 'geometric']:
    run_clustering_evaluation(joined_df, window_sizes, method='ahc', moving_average=True, return_mode=return_mode, df_input_name=df_input_name)

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
c:\Users\Kirill\Documents\Projects\

In [20]:
folder_path = 'silhouette_dfs/'
dfs_all_sil = pd.DataFrame()

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if '456' not in filename: 
        df = pd.read_csv(file_path)
        df['filename'] = filename
        dfs_all_sil = pd.concat([dfs_all_sil, df], axis=0)

dfs_all_sil = dfs_all_sil.drop(columns=['Unnamed: 0'])
dfs_all_sil['df_mode'] = dfs_all_sil['filename'].apply(lambda x: x.split('-')[1].split('_')[1])


In [21]:
dfs_all_sil.sort_values(by='silhouette_score', ascending=False)

,clusters,silhouette_score,method,linkage,return_mode,n_init,smoothing,window_size/span,filename,inertia,df_mode
6,6,0.604888,ahc,average,arithmetic,3,moving_average,30,ahc_moving_average_geometric_windowsize-30_wee...,NaN,weekly.csv
0,4,0.579029,kmeans,NaN,arithmetic,3,moving_average,30,kmeans_moving_average_geometric_windowsize-30_...,1.283079,weekly.csv
7,7,0.574291,ahc,average,arithmetic,3,moving_average,30,ahc_moving_average_geometric_windowsize-30_wee...,NaN,weekly.csv
4,4,0.568835,ahc,average,arithmetic,3,moving_average,30,ahc_moving_average_geometric_windowsize-30_wee...,NaN,weekly.csv
5,5,0.564094,ahc,average,arithmetic,3,moving_average,30,ahc_moving_average_geometric_windowsize-30_wee...,NaN,weekly.csv
...,...,...,...,...,...,...,...,...,...,...,...
2,6,-0.135714,ahc,single,arithmetic,3,moving_average,30,ahc_moving_average_arithmetic_windowsize-30_we...,NaN,weekly.csv
3,7,-0.138355,ahc,single,arithmetic,3,moving_average,30,ahc_moving_average_arithmetic_windowsize-30_we...,NaN,weekly.csv
1,5,-0.315591,ahc,single,arithmetic,3,moving_average,60,ahc_moving_average_geometric_windowsize-60_3da...,NaN,3day.csv
2,6,-0.338285,ahc,single,arithmetic,3,moving_average,60,ahc_moving_average_geometric_windowsize-60_3da...,NaN,3day.csv


In [22]:
x = dfs_all_sil.groupby(by=['method', 'window_size/span'])['silhouette_score'].mean()
pd.DataFrame(x)

silhouette_score
method window_size/span                  
ahc    3                         0.176464
       7                         0.220252
       10                        0.203185
       14                        0.202288
       21                        0.186192
       30                        0.230353
       60                        0.204818
kmeans 3                         0.033833
       7                         0.071319
       10                        0.084753
       14                        0.118078
       21                        0.197587
       30                        0.307431
       60                        0.229079

In [23]:
pd.pivot_table(dfs_all_sil, values="silhouette_score", index=["window_size/span", 'clusters'], columns=["method"])

method                          ahc    kmeans
window_size/span clusters                    
3                4         0.222333  0.036113
                 5         0.187491  0.027186
                 6         0.168248  0.040836
                 7         0.127785  0.031198
7                4         0.278616  0.087296
                 5         0.242141  0.072707
                 6         0.208037  0.065981
                 7         0.152215  0.059292
10               4         0.280568  0.097647
                 5         0.220884  0.082574
                 6         0.173305  0.079992
                 7         0.137984  0.078798
14               4         0.280319  0.123920
                 5         0.220078  0.129105
                 6         0.175287  0.102750
                 7         0.133466  0.116535
21               4         0.235797  0.209115
                 5         0.207486  0.190430
                 6         0.173345  0.205476
                 7         0.128139  0.185325
30               4         0.277488  0.350091
                 5         0.236924  0.306865
                 6         0.217913  0.283211
                 7         0.189087  0.289557
60               4         0.282158  0.244157
                 5         0.201130  0.253826
                 6         0.179133  0.198024
                 7         0.156851  0.220309